In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Function to generate data
def generate_data(data_type='quadratic'):
    np.random.seed(np.random.randint(0, 1000))
    x = np.random.uniform(-10, 10, 100)
    if data_type == 'quadratic':
        y = 3 * x**2 + 2 * x + 1 + np.random.normal(0, 10, 100)
    elif data_type == 'linear':
        y = 2 * x + 1 + np.random.normal(0, 5, 100)
    elif data_type == 'sinusoidal':
        y = 10 * np.sin(x) + np.random.normal(0, 2, 100)
    elif data_type == 'exponential':
        y = np.exp(0.2 * x) + np.random.normal(0, 2, 100)
    else:  # random
        y = np.random.normal(0, 20, 100)
    return x, y

# Initial data generation
x, y = generate_data()

# Split into train and validation sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, random_state=0)

# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self, hidden_dims):
        super(SimpleNN, self).__init__()
        layers = []
        input_dim = 1
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, int(hidden_dim)))
            layers.append(nn.ReLU())
            input_dim = int(hidden_dim)
        layers.append(nn.Linear(input_dim, 1))
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

# Function to visualize the network and data
def visualize_network(model, x_train, y_train, x_val, y_val, train_loss_history, val_loss_history, epochs):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))
    
    # Plot network output
    ax1.scatter(x_train, y_train, color='blue', label='Train set')
    ax1.scatter(x_val, y_val, color='red', label='Validation set')
    
    # Generate predictions
    x_range = np.linspace(-10, 10, 1000).reshape(-1, 1)
    x_range_tensor = torch.tensor(x_range, dtype=torch.float32)
    y_pred = model(x_range_tensor).detach().numpy()
    
    ax1.plot(x_range, y_pred.flatten(), color='green', label='NN output')
    ax1.legend()
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_title('Neural Network Output')
    
    # Plot loss curves
    ax2.plot(train_loss_history[:epochs], label='Training Loss')
    ax2.plot(val_loss_history[:epochs], label='Validation Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.set_title('Loss Curves')
    ax2.set_yscale('log')
    ax2.legend()
    
    plt.tight_layout()
    return fig

# Function to train the model
def train_model(hidden_dims, learning_rate, epochs):
    model = SimpleNN(hidden_dims)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    x_train_tensor = torch.tensor(x_train.reshape(-1, 1), dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
    x_val_tensor = torch.tensor(x_val.reshape(-1, 1), dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.reshape(-1, 1), dtype=torch.float32)
    
    train_loss_history = []
    val_loss_history = []
    weight_history = []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(x_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        train_loss_history.append(loss.item())
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(x_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor)
            val_loss_history.append(val_loss.item())
        
        # Save weights
        weights = [param.data.clone() for param in model.parameters()]
        weight_history.append(weights)
    
    return model, train_loss_history, val_loss_history, weight_history

# Initialize variables to store the current model and histories
current_model = None
current_train_loss_history = None
current_val_loss_history = None
current_weight_history = None
current_hidden_dims = None
current_learning_rate = None
current_data_type = None

# Function to update the plot and weights visualization
def update_plot(epochs, learning_rate, hidden_dims, test_split, data_type):
    global current_model, current_train_loss_history, current_val_loss_history, current_weight_history, current_hidden_dims, current_learning_rate, current_data_type, x, y, x_train, x_val, y_train, y_val
    
    clear_output(wait=True)
    hidden_dims = parse_hidden_dims(hidden_dims)
    
    # Check if we need to regenerate data
    if current_data_type != data_type:
        x, y = generate_data(data_type)
        current_data_type = data_type
        current_model = None  # Force retraining
    
    # Update train-test split if necessary
    if x_val.size / x.size != test_split or current_model is None:
        x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=test_split, random_state=0)
        current_model = None  # Force retraining
    
    # Check if we need to retrain the model
    if (current_model is None or 
        current_hidden_dims != hidden_dims or 
        current_learning_rate != learning_rate):
        current_model, current_train_loss_history, current_val_loss_history, current_weight_history = train_model(hidden_dims, learning_rate, 1000)  # Train for max epochs
        current_hidden_dims = hidden_dims
        current_learning_rate = learning_rate
    
    # Use the current model and histories
    model = current_model
    train_loss_history = current_train_loss_history[:epochs]
    val_loss_history = current_val_loss_history[:epochs]
    weight_history = current_weight_history[:epochs]
    
    # Update model weights based on the current epoch
    with torch.no_grad():
        for param, saved_weight in zip(model.parameters(), current_weight_history[epochs-1]):
            param.copy_(saved_weight)
    
    model.eval()
    
    # Visualize the network output and loss curves
    fig = visualize_network(model, x_train, y_train, x_val, y_val, train_loss_history, val_loss_history, epochs)
    plt.show()

# Create interactive UI with ipywidgets
epochs_slider = widgets.IntSlider(value=100, min=10, max=1000, step=10, description='Epochs:')
learning_rate_slider = widgets.FloatLogSlider(value=0.01, base=10, min=-3, max=-1, step=0.1, description='Learning Rate:')
hidden_dims_input = widgets.Text(value='2', description='Hidden Dims:')
test_split_slider = widgets.FloatSlider(value=0.1, min=0.05, max=0.5, step=0.05, description='Test Split:')
data_type_dropdown = widgets.Dropdown(
    options=['quadratic', 'linear', 'sinusoidal', 'exponential', 'random'],
    value='quadratic',
    description='Data Type:'
)

def parse_hidden_dims(hidden_dims_str):
    try:
        return [int(dim) for dim in hidden_dims_str.split(',')]
    except ValueError:
        return [2]  # Default value if parsing fails

ui = widgets.VBox([epochs_slider, learning_rate_slider, hidden_dims_input, test_split_slider, data_type_dropdown])
out = widgets.interactive_output(update_plot, {
    'epochs': epochs_slider,
    'learning_rate': learning_rate_slider,
    'hidden_dims': hidden_dims_input,
    'test_split': test_split_slider,
    'data_type': data_type_dropdown
})

display(ui, out)